In [5]:
import numpy as np
import pyphs
import matplotlib.pyplot as plt
import sympy as sp

In [86]:
class SubGlottalCavity(pyphs.Core):
    """
    Junction component for the subglottal region.
    Flow coming from trachea: (Psub, Qsub, receptor convention)
    separates into the flows due to the motion of the folds
    (Qsubl, Psubl, generator convention) and (Qsubr, Psubr, generator convention)
    and the flow entering the glottis channel:
    (Qu, Ptotu, generator convention)
    """
    def __init__(self, label='Subglottal cavity', **kwargs):
        pyphs.Core.__init__(self, label)
        # Input variables
        Psub, Qsubl, Qsubr, Qu = self.symbols('P_sub Q_sub_l Q_sub_r Q_u')
        # Output variables
        Qsub, Psubl, Psubr, Ptotu = self.symbols('Q_sub P_sub_l P_sub_r P_totu')

        self.add_ports(Psub, -Qsub)
        self.add_ports(Qsubl, Psubl)
        self.add_ports(Qsubr, Psubr)
        self.add_ports(Qu, Ptotu)
        self.set_Jyy([[0, -1, -1, -1],
                      [1, 0, 0, 0],
                      [1, 0, 0, 0],
                      [1, 0, 0, 0]])


class SupraGlottalCavity(pyphs.Core):
    """
    Junction component for the supraglottal region.
    Flow coming from the glottis: (Pd, Qd, receptor convention)
    sums up with the flows due to the motion of the folds
    (Qsupl, Psupl, receptor convention) and (Qsupr, Psupr, receptor convention)
    and gets in the acoustic resonator:
    (Qac, Pac, generator convention)
    """
    def __init__(self, label='Supraglottal cavity', **kwargs):
        pyphs.Core.__init__(self, label)
        # Input variables
        Qac, Qsupl, Qsupr, Qd = self.symbols('Q_ac Q_sup_l Q_sup_r Q_d')
        # Output variables
        Pac, Psupl, Psupr, Pd = self.symbols('P_ac P_sup_l P_sup_r P_d')

        self.add_ports(Pac, Qac)
        self.add_ports(Qsupl, -Psupl)
        self.add_ports(Qsupr, -Psupr)
        self.add_ports(Qd, -Pd)
        self.set_Jyy([[0, 1, 1, 1],
                      [-1, 0, 0, 0],
                      [-1, 0, 0, 0],
                      [-1, 0, 0, 0]])

class VocalFoldBase(pyphs.Core):
    """
    Abstract class for vocal fold models. Derived classes must have (at least) the following ports:
    - a flux controlled port to the glottis: (v, F, generator convention) 
    - an effort controlled port to the subglottal cavity: (Psub, Qsub, receiver convention)
    - an effort controlled port to the supraglottal cavity: (Psup, Qsup, generator convention)
    """
    pass

class VocalFold_SingleMass(VocalFoldBase):
    """
    Single mass model for vocal fold containing
    - a bulk single dof oscillator (mass m, damping r, stiffness k)
    - an elastic cover (stiffness kappa)
    The variables are:
    - the momentum pi of the mass
    - the elongation xi of the bulk spring
    - the elongation zeta of the cover spring
    - the dissipation variable w of the dashpot
    
    H = 1/2 * (pi**2/m + k*xi**2 + kappa*zeta**2)
    z(w) = rw
    """
    def __init__(self, label='',  **kwargs):
        pyphs.Core.__init__(self, label)

        if label:
            helper = lambda *elements: self.symbols(['%s_%s' % (el, label) for el in elements])
        else:
            helper = lambda *elements: self.symbols(' '.join(elements))
        # Parameters
        m, k, kappa, r, Ssub, Ssup = helper('m', 'k', 'kappa', 'r', 'S_sub', 'S_sup')
        # Internal and dissipation variables
        pi, xi, zeta, w = helper('pi', 'xi', 'zeta', 'w')
        # Input variables
        v, Psub, Psup = helper('v', 'P_sub', 'P_sup')
        # Output variables
        Fp, Qsub, Qsup = helper('F', 'Q_sub', 'Q_sup')

        H = (pi**2/m + k*xi**2 + kappa*zeta**2)/2

        self.add_storages([pi, xi, zeta], H)
        self.add_dissipations(w, r*w)
        self.add_ports(Psub, -Qsub)
        self.add_ports(Psup, Qsup)
        self.add_ports(v, Fp)

        self.set_Jxx([[0, -1, 1], [1, 0, 0], [-1, 0, 0]])
        self.set_Jxw([-1, 0, 0])
        self.set_Jxy([[-Ssub, -Ssup, 0], [0, 0, 0], [0, 0, 1]])
        self.reduce_z()

        self.subs.update({self.symbols(k + '_' + label): v
                          for k, v in kwargs.items()})

class VocalTractBase(pyphs.Core):
    """
    Abstract class for the vocal tract model. Derived classes must have (at least) the following port:
    - the flux controlled port on the upstream open surfaces:
      (Q_ac, P_ac, receiver convention)
    """
    pass


class ModalVocalTract(VocalTractBase):
    """
    Vocal tract as seen from its input section.
    THe input impedance is described modally using:
    - its natural angular frequencies omega_n
    - their damping q_n
    - their complex amplitude a_n
    """
    def __init__(self, label='vt', nmodes=1, **kwargs):
        pyphs.Core.__init__(self, label)
        # Parameters
        an = self.symbols('a:%d' % nmodes)
        omegan = self.symbols('omega:%d' % nmodes)
        qn = self.symbols('q:%d' % nmodes)
        # Internal variables
        Xn = self.symbols('X:%d' % nmodes)
        Yn = self.symbols('Y:%d' % nmodes)
        # Dissipation variables
        wn = self.symbols('w:%d' % nmodes)
        # Ports
        Qac, Pac = self.symbols('Q_ac P_ac')

        Hac = sum([a*x**2 + w**2*y**2/a for x, y, a, w in zip(Xn, Yn, an, omegan)]) / 2

        self.add_storages(list(Xn) + list(Yn), Hac)
        self.add_dissipations(wn, [q*omega*w/a for q, omega, a, w in zip(qn, omegan, an, wn)])
        self.add_ports(Qac, -Pac)

        On, In = sp.zeros(nmodes), sp.eye(nmodes)
        Jxx = On.row_join(-In).col_join(In.row_join(On))
        Jxw = -In.col_join(On)
        Jxy = sp.ones(nmodes, 1).col_join(sp.zeros(nmodes, 1))
        self.set_Jxx(Jxx)
        self.set_Jxw(Jxw)
        self.set_Jxy(Jxy)

        self.subs.update({self.symbols(k): v for k, v in kwargs.items()})


class GlottalFlowBase(pyphs.Core):
    """
    Abstract class for the glottal flow model. Derived classes must have (at least) the following ports:
    - two effort controlled ports to the left and right folds:
      (F_l, v_l, receiver convention) and (F_r, v_r, receiver convention)
    - the effort controlled port on the upstream open surfaces:
      (P_totu, Q_u, receiver convention)
    - the effort controlled port at the end of the mixing region:
      (P_d, Q_d, generator convention)
    """
    pass

class GlottalFlow_straightchannel_velocity(GlottalFlowBase):
    """
    Glottal flow model based on a parametrization of the velocity field using
    - the mean axial velocity V_x
    - the mean transverse velocity V_y
    - the transverse expansion velocity V_exp=dh/dt
    - the channel height h+h0

    The parameters are:
    - the density of the fluid: rho
    - the width of the channel (out-of-plane z dimension): L0
    - the half-length of the channel: ell0
    - the height of the channel at rest: hr

    The flow is assumed to be potential, incompressible and inviscid.
    However the model includes the existence of a jet downstream of the channel,
    the jet dissipating the kinetic energy of the flow without recovering pressure.
    This is modelled as a dissipative component with variable w_turb.
    """
    def __init__(self, label='glottis', **kwargs):
        pyphs.Core.__init__(self, label)

        helper = lambda *elements: self.symbols(['%s' % el for el in elements])
        # Parameters
        rho, L0, ell0, hr = self.symbols('rho L_0 ell_0 h_r')
        # Internal and dissipation variables
        Vx, Vy, Vexp, h, wturb = self.symbols('V_x V_y V_exp h w_turb')
        # Input variables
        Ptotu, Pd, Fl, Fr = self.symbols('P_totu P_d F_l F_r')
        # Output variables
        Qu, Qd, vl, vr = self.symbols('Q_u Q_d v_l v_r')

        htotal = hr + h
        mu0 = rho * L0 * 2*ell0
        m = mu0 * htotal
        m33 = m/12 * (1 + 4*ell0**2/htotal**2)
        H = m/2 * (Vx**2 + Vy**2) + m33/2*Vexp**2
        from sympy import Abs
        zturb = rho/2 * ((wturb + Abs(wturb))/(2 * L0 * htotal))**2

        # Gyrators
        wg1, wg2, wg3, wg4= self.symbols('wg1 wg2 wg3 wg4')
        factor1 = 1/m
        zg1 = wg2*factor1
        zg2 = -wg1*factor1
        
        factor2 = 1/m33
        zg3 = wg4*factor2
        zg4 = -wg3*factor2

        self.add_storages([Vx, Vy, Vexp, h], H)
        self.add_dissipations([wturb, wg1, wg2, wg3, wg4], [zturb, zg1, zg2, zg3, zg4])
        self.add_ports(Ptotu, -Qu)
        self.add_ports(Pd, Qd)
        self.add_ports(Fl, -vl)
        self.add_ports(Fr, -vr)

        self.set_Jxx([[0, 0, 0, 0],
                      [0, 0, 0, 0],
                      [0, 0, 0, 0],
                      [0, 0, 0, 0]])
        self.set_Jxw([[-L0/mu0, 0, 0 , 0, 0],
                     [0, 1, 0, 0, 0],
                     [0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 1]])
        self.set_Jxy([[L0/mu0, -L0/mu0, 0, 0],
                      [0, 0, 0, 0],
                      [0, 0, 0, 0],
                      [0, 0, 0, 0]])
        self.set_Jww([[0, 0, 0, 0, -ell0*L0],
                      [0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0],
                      [ell0*L0, 0, 0, 0, 0]])
        self.set_Jyw([[0, 0, 0 , 0, -ell0*L0],
                     [0, 0, 0, 0, -ell0*L0],
                     [0, 0, 1, 0, +0.5],
                     [0, 0, -1, 0, +0.5]])

        self.subs.update({self.symbols(k): v for k, v in kwargs.items()})


In [87]:
class VocalApparatusBase(pyphs.Core):
    """
    Abstract class for the vocal apparatus model. Derived classes must have (at least) the following port:
    - the effort controlled port on the upstream open surfaces:
      (P_sub, Q_sub, receiver convention)
    """
    pass


class VocalApparatus_with_height(VocalApparatusBase):
    """
    Minimal complete vocal apparatus accounting for
    - two vocal folds (single dof oscillator for bulk + elastic cover)
    - glottal flow between two parallel moving walls (momentum_h0 formulation)
    - vocal tract (modal representation)
    Controlled by the subglottal pressure
    """
    def __new__(cls, left_fold=None, right_fold=None, glottis=None, vocal_tract=None, *args, **kwargs):
        # Left vocal fold
        if left_fold is None:
            left_fold = VocalFold_SingleMass(label='l', m=2e-4, r=0.01, k=100, kappa=300, Ssup=1e-6, Ssub=1e-4)
        elif isinstance(left_fold, dict):
            left_fold = VocalFold_SingleMass(label='l', **left_fold)
        else:
            assert isinstance(left_fold, VocalFoldBase)

        # Right vocal fold
        if right_fold is None:
            right_fold = VocalFold_SingleMass(label='r', m=2e-4, r=0.01, k=100, kappa=300, Ssup=1e-6, Ssub=1e-4)
        elif isinstance(right_fold, dict):
            right_fold = VocalFold_SingleMass(label='r', **right_fold)
        else:
            assert isinstance(right_fold, VocalFoldBase)

        # Glottis
        if glottis is None:
            # Default is strong adduction
            glottis = GlottalFlow_straightchannel_velocity(rho=1.3, L0=11e-3, ell0=2e-3, hr=1e-4, h0=1e-4)
        elif isinstance(glottis, dict):
            glottis = GlottalFlow_straightchannel_velocity(**glottis)
        else:
            assert isinstance(glottis, GlottalFlowBase)
        
        # Vocal tract
        if vocal_tract is None:
            # Default is a single mode at 640Hz (low damping and high amplitude to ease the oscillation)
            vocal_tract = ModalVocalTract(nmodes=1, a0=1e8, w0=2*np.pi*640, q0=.04)
        elif isinstance(vocal_tract, dict):
            vocal_tract = ModalVocalTract(**vocal_tract)
        else:
            assert isinstance(vocal_tract, ModalVocalTract)
            vocal_tract = vocal_tract.__copy__() 
        #vocal_tract.reduce_z()

        SubC = SubGlottalCavity()
        SupC = SupraGlottalCavity()
        obj = left_fold + right_fold + glottis + vocal_tract + SubC + SupC
        obj.__class__ = cls
        obj.lfold = left_fold
        obj.rfold = right_fold
        obj.glottis = glottis
        obj.vt = vocal_tract
        obj.subcavity = SubC
        obj.supcavity = SupC
        return obj

    def __init__(self, *args, **kwargs):
        for var in ['v_r', 'v_l', 'P_totu', 'P_d', 'P_ac',
                    'P_sub_r', 'P_sub_l', 'P_sup_r', 'P_sup_l']:
            self.connect_components_by_label(var)
        self.connect()

    def connect_components_by_label(self, var):
        """
        Connects two ports of subcomponents assuming one is receptor
        and the other one is generator.
        """
        i1 = [str(el) for el in self.u].index(var)
        try:
            # var is:
            # * the input of the component with receptor convention
            # * the output of the component with generator convention
            i2 = [str(el) for el in self.y].index(var)
            # Swap to the dual variable
            i1, i2 = i2, i1
        except ValueError:
            # var is:
            # * the output of the component with receptor convention
            # * the input of the component with generator convention
            i2 = [str(el).strip('-') for el in self.y].index(var)

        # self.u[i2] = +self.y[i1] * alpha
        # self.u[i1] = -self.y[i2] * alpha
        self.add_connector([i1, i2], alpha=1)

In [76]:
Vapp = VocalApparatus_with_height()

In [77]:
Vapp.M

Matrix([
[   -r_l, -1,   1,       0,  0,   0,                0,  0,  0,  0,         -S_sup_l,  0,                0, 0,  0, 0,          0,  0,        -S_sub_l],
[      1,  0,   0,       0,  0,   0,                0,  0,  0,  0,                0,  0,                0, 0,  0, 0,          0,  0,               0],
[     -1,  0,   0,       0,  0,   0,                0,  0,  0,  0,                0,  0,                0, 0, -1, 0,       -0.5,  0,               0],
[      0,  0,   0,    -r_r, -1,   1,                0,  0,  0,  0,         -S_sup_r,  0,                0, 0,  0, 0,          0,  0,        -S_sub_r],
[      0,  0,   0,       1,  0,   0,                0,  0,  0,  0,                0,  0,                0, 0,  0, 0,          0,  0,               0],
[      0,  0,   0,      -1,  0,   0,                0,  0,  0,  0,                0,  0,                0, 0,  1, 0,       -0.5,  0,               0],
[      0,  0,   0,       0,  0,   0,                0,  0,  0,  0, -1/(2*ell_0*rho), 

In [84]:
Vfold = VocalFold_SingleMass()

In [85]:
Vfold.M

Matrix([
[   -r, -1,  1, -S_sub, -S_sup, 0],
[    1,  0,  0,      0,      0, 0],
[   -1,  0,  0,      0,      0, 1],
[S_sub,  0,  0,      0,      0, 0],
[S_sup,  0,  0,      0,      0, 0],
[    0,  0, -1,      0,      0, 0]])

In [88]:
Gflow = GlottalFlow_straightchannel_velocity()

In [80]:
sp.simplify(Gflow.dxH()[3])

-L_0*V_exp**2*ell_0**3*rho/(3*(h + h_r)**2) + L_0*V_exp**2*ell_0*rho/12 + L_0*V_x**2*ell_0*rho + L_0*V_y**2*ell_0*rho

In [94]:
Gflow.dxH()[3].simplify()

-L_0*V_exp**2*ell_0**3*rho/(3*(h + h_r)**2) + L_0*V_exp**2*ell_0*rho/12 + L_0*V_x**2*ell_0*rho + L_0*V_y**2*ell_0*rho

In [69]:
Gflow.M

Matrix([
[               0,  0,  0,  0, -1/(2*ell_0*rho), 0,  0, 0,          0, 1/(2*ell_0*rho), -1/(2*ell_0*rho),    0,    0],
[               0,  0,  0,  0,                0, 1,  0, 0,          0,               0,                0,    0,    0],
[               0,  0,  0,  0,                0, 0,  0, 1,          0,               0,                0,    0,    0],
[               0,  0,  0,  0,                0, 0,  0, 0,          1,               0,                0,    0,    0],
[ 1/(2*ell_0*rho),  0,  0,  0,                0, 0,  0, 0, -L_0*ell_0,               0,                0,    0,    0],
[               0, -1,  0,  0,                0, 0,  0, 0,          0,               0,                0,    0,    0],
[               0,  0,  0,  0,                0, 0,  0, 0,          0,               0,                0,   -1,    1],
[               0,  0, -1,  0,                0, 0,  0, 0,          0,               0,                0,    0,    0],
[               0,  0,  0, -1,        L